# Atividade 02: Ingestão e ETL com linguagem de programação (Python)

- Utilizar linguagem de programação Python para ingestão e tratamento de dados.
    - Pacotes adicionais podem ser utilizados (exceto Spark)
    - Tratamento de dados não deve ser realizado via SQL
- Realizar a ingestão de todas as base de dados em um banco de dados relacional open source. Pode ser utilizado qualquer banco de dado sendo algumas sugestões:
    - MySQL
    - Postgre
    - ClickHouse
- Gerar uma tabela final com os dados tratados e unidos.
    - O tratamento de dados deve ser realizado através da linguagem de programação Python
- Adicionar as seguintes camadas de processamento, dentro do próprio banco de dados ou em disco local. A Camada Delivery deve obrigatoriamente ter estar também no formato de uma tabela final dentro do banco de dados relacional:
    - RAW – formato dos dados livre
    - Trusted – formato de dados em Parquet ou ORC or AVRO (indicado Parquet)
    - Delivery– formato de dados em Parquet ou ORC or AVRO (indicado Parquet)

## To Do

- raw 
    - salvar em parquet (ok)
    - salvar rwzd_{tabela} (ok)
    - setar schema de str
    - criar as pastas pros parquets
- trusted
    - ler da raw (ok)
    - criar uma chave artificial em todas as tabelas trusted a partir do cálculo de hash da coluna que contém o nome da instituição financeira, porém, removendo os espaços em branco e deixando todas as letras em caixa baixa e removendo a acentuação.
    - aplicar data quality
        - trocar os nomes das colunas para ingles (pegar do arquivo de DDL)
        - trocar os nomes das colunas para snakecase
        - mudar tipos
        - outras transformacoes
            - remover string "- prudential"
            - remover coluna unnamed
- delivery
    - tabela consolidada
    - agregação

---
## Library Imports

In [7]:
import pandas as pd
import os

---
## Raw

### Banks file

In [8]:
banks = pd.read_csv('./trzd/Bancos/EnquadramentoInicia_v2_alterado_manualmente_revisado.csv', sep='\t', encoding='utf8')

In [9]:
banks.head()

,Segmento,CNPJ,Nome
0,S1,0,BANCO DO BRASIL - PRUDENCIAL
1,S1,60746948,BRADESCO - PRUDENCIAL
2,S1,30306294,BTG PACTUAL - PRUDENCIAL
3,S1,360305,CAIXA ECONOMICA FEDERAL - PRUDENCIAL
4,S1,60872504,ITAU - PRUDENCIAL


In [11]:
banks.to_parquet('./raw/rwzd_banks.parquet', engine='pyarrow', compression='snappy')

### Employees file

In [13]:
employees = pd.read_csv('./trzd/Empregados/glassdoor_consolidado_join_match_v2.csv', sep='|', encoding='utf-8')

In [14]:
employees.head(2)

,employer_name,reviews_count,culture_count,salaries_count,benefits_count,employer-website,employer-headquarters,employer-founded,employer-industry,employer-revenue,...,Diversidade e inclusão,Qualidade de vida,Alta liderança,Remuneração e benefícios,Oportunidades de carreira,Recomendam para outras pessoas(%),Perspectiva positiva da empresa(%),Segmento,Nome,match_percent
0,BNP Paribas,13000,4100,20000,3600,http://www.group.bnpparibas,"Paris, França",2000.0,/Explorar/melhores-serviços-bancários-e-de-cré...,Mais de US$ 10 bilhões,...,4.0,3.8,3.4,3.4,3.5,77.0,63.0,S3,BNP PARIBAS,100
1,BTG Pactual,1600,683,2800,635,http://www.btgpactual.com,"São Paulo, Brasil",NaN,/Explorar/melhores-gestão-de-ativos-e-investim...,Desconhecida/não se aplica,...,3.5,2.8,3.7,4.4,4.2,78.0,73.0,S1,BTG PACTUAL,100


In [16]:
employees.to_parquet('./raw/rwzd_employees.parquet', engine='pyarrow', compression='snappy')

### Claims file

In [17]:
list_df_claims = []
dir = './trzd/Reclamações'

for file in os.listdir(dir):
    full_path = os.path.join(dir, file)
    print(f'Processing file: {full_path}')
    list_df_claims.append(pd.read_csv(full_path, sep=';', encoding='utf-8'))


Processing file: ./trzd/Reclamações\2021_tri_01.csv
Processing file: ./trzd/Reclamações\2021_tri_02.csv
Processing file: ./trzd/Reclamações\2021_tri_03.csv
Processing file: ./trzd/Reclamações\2021_tri_04.csv
Processing file: ./trzd/Reclamações\2022_tri_01.csv
Processing file: ./trzd/Reclamações\2022_tri_03.csv
Processing file: ./trzd/Reclamações\2022_tri_04.csv


In [18]:
claims = pd.concat(list_df_claims)
claims.head(2)

,Ano,Trimestre,Categoria,Tipo,CNPJ IF,Instituição financeira,Índice,Quantidade de reclamações reguladas procedentes,Quantidade de reclamações reguladas - outras,Quantidade de reclamações não reguladas,Quantidade total de reclamações,Quantidade total de clientes – CCS e SCR,Quantidade de clientes – CCS,Quantidade de clientes – SCR,Unnamed: 14
0,2021,1º,Grupo Secundário,Conglomerado,,ABC-BRASIL (conglomerado),,2,3,4,9,26230,24698,3810,NaN
1,2021,1º,Grupo Secundário,Conglomerado,,AGIBANK (conglomerado),"54,79",58,140,73,271,1058431,790848,693843,NaN


In [43]:
# este campo estava com strings e numeros. Como haviam strings vazias, converti a coluna para str a fim de discutirmos depois como tratar na etapa Trusted
claims['Quantidade total de clientes – CCS e SCR'] = claims['Quantidade total de clientes – CCS e SCR'].astype(str)

In [44]:
claims.to_parquet('./raw/rwzd_claims.parquet', engine='pyarrow', compression='snappy')

---
## Trusted

### Banks

In [222]:
banks = pd.read_parquet('./raw/rwzd_banks.parquet', engine='pyarrow')

In [223]:
#  rever
banks.rename(columns={
    'Segmento': 'segment',
    'CNPJ': 'cnpj',
    'Nome': 'name'
}, inplace = True)

In [229]:
banks[['name', 'cnpj']][banks['name'].str.contains('ITAU')]

,name,cnpj
4,ITAU - PRUDENCIAL,60872504
1458,BANCO ITAU S/A,33058660
1459,BANCO ITAUBANK S.A.,60394079
1473,ITAU UNIBANCO HOLDING S.A.,60872504


In [125]:
banks.head()

,segment,cnpj,name
0,S1,0,BANCO DO BRASIL
1,S1,60746948,BRADESCO
2,S1,30306294,BTG PACTUAL
3,S1,360305,CAIXA ECONOMICA FEDERAL
4,S1,60872504,ITAU


### Employees

In [126]:
employees = pd.read_parquet('./raw/rwzd_employees.parquet', engine='pyarrow')

In [127]:
employees.columns

Index(['employer_name', 'reviews_count', 'culture_count', 'salaries_count',
       'benefits_count', 'employer-website', 'employer-headquarters',
       'employer-founded', 'employer-industry', 'employer-revenue', 'url',
       'Geral', 'Cultura e valores', 'Diversidade e inclusão',
       'Qualidade de vida', 'Alta liderança', 'Remuneração e benefícios',
       'Oportunidades de carreira', 'Recomendam para outras pessoas(%)',
       'Perspectiva positiva da empresa(%)', 'Segmento', 'Nome',
       'match_percent'],
      dtype='object')

In [219]:
# rever
# tratamento no nome das colunas para snake case
employees.columns = employees.columns.map(lambda x: x.replace(" ", "_").replace("-", "_").lower())

In [129]:
# mudança do nome das colunas para o inglês
employees.rename(columns={
    'geral': 'general',
    'cultura_e_valores': 'culture_and_values',
    'diversidade_e_inclusão': 'diversity_and_inclusion',
    'qualidade_de_vida': 'quality_of_life',
    'alta_liderança': 'senior_leadership',
    'remuneração_e_benefícios': 'compensation_and_benefits',
    'oportunidades_de_carreira': 'career_opportunities',
    'recomendam_para_outras_pessoas(%)': 'recommend_to_others',
    'perspectiva_positiva_da_empresa(%)': 'positive_company_outlook',
    'segmento': 'segment',
    'nome': 'name'
}, inplace=True)

In [217]:
# conversão da coluna employer_founded para inteiro
#  rever, talvez nao precise setando o schema na raw
employees['employer_founded'] = employees['employer_founded'].fillna(0)
employees['employer_founded'] = employees['employer_founded'].astype(str)

In [131]:
employees.head(2)

,employer_name,reviews_count,culture_count,salaries_count,benefits_count,employer_website,employer_headquarters,employer_founded,employer_industry,employer_revenue,...,diversity_and_inclusion,quality_of_life,senior_leadership,compensation_and_benefits,career_opportunities,recommend_to_others,positive_company_outlook,segment,name,match_percent
0,BNP Paribas,13000,4100,20000,3600,http://www.group.bnpparibas,"Paris, França",2000,/Explorar/melhores-serviços-bancários-e-de-cré...,Mais de US$ 10 bilhões,...,4.0,3.8,3.4,3.4,3.5,77.0,63.0,S3,BNP PARIBAS,100
1,BTG Pactual,1600,683,2800,635,http://www.btgpactual.com,"São Paulo, Brasil",0,/Explorar/melhores-gestão-de-ativos-e-investim...,Desconhecida/não se aplica,...,3.5,2.8,3.7,4.4,4.2,78.0,73.0,S1,BTG PACTUAL,100


### Claims

In [203]:
claims = pd.read_parquet('./raw/rwzd_claims.parquet', engine='pyarrow')

In [204]:
claims.head(2)

,Ano,Trimestre,Categoria,Tipo,CNPJ IF,Instituição financeira,Índice,Quantidade de reclamações reguladas procedentes,Quantidade de reclamações reguladas - outras,Quantidade de reclamações não reguladas,Quantidade total de reclamações,Quantidade total de clientes – CCS e SCR,Quantidade de clientes – CCS,Quantidade de clientes – SCR,Unnamed: 14
0,2021,1º,Grupo Secundário,Conglomerado,,ABC-BRASIL (conglomerado),,2,3,4,9,26230,24698,3810,NaN
1,2021,1º,Grupo Secundário,Conglomerado,,AGIBANK (conglomerado),"54,79",58,140,73,271,1058431,790848,693843,NaN


In [205]:
# remoção de coluna sem dados
claims = claims.drop(columns=['Unnamed: 14'])

In [206]:
claims.columns

Index(['Ano', 'Trimestre', 'Categoria', 'Tipo', 'CNPJ IF',
       'Instituição financeira', 'Índice',
       'Quantidade de reclamações reguladas procedentes',
       'Quantidade de reclamações reguladas - outras',
       'Quantidade de reclamações não reguladas',
       'Quantidade total de reclamações',
       'Quantidade total de clientes – CCS e SCR',
       'Quantidade de clientes – CCS', 'Quantidade de clientes – SCR'],
      dtype='object')

In [207]:
# rever
# tratamento no nome das colunas para snake case
claims.columns = claims.columns.map(lambda x: x.lower().replace(' ', '_').replace('–', '_').replace('-', '_').replace('___', '_'))

In [208]:
# mudança do nome das colunas para o inglês
claims.rename(columns={
    'ano': 'year',
    'trimestre': 'quarter',
    'categoria': 'category',
    'tipo': 'type',
    'cnpj_if': 'cnpj',
    'instituição_financeira': 'financial_institution',
    'índice': 'index',
    'quantidade_de_reclamações_reguladas_procedentes': 'number_of_regulated_proceeding_complaints',
    'quantidade_de_reclamações_reguladas_outras': 'number_of_regulated_other_complaints',
    'quantidade_de_reclamações_não_reguladas': 'number_of_non_regulated_complaints',
    'quantidade_total_de_reclamações': 'total_number_of_complaints',
    'quantidade_total_de_clientes_ccs_e_scr': 'total_number_of_clients_ccs_and_scr',
    'quantidade_de_clientes_ccs': 'number_of_clients_ccs',
    'quantidade_de_clientes_scr': 'number_of_clients_scr'
}, inplace=True)

In [209]:
claims.head(2)

,year,quarter,category,type,cnpj,financial_institution,index,number_of_regulated_proceeding_complaints,number_of_regulated_other_complaints,number_of_non_regulated_complaints,total_number_of_complaints,total_number_of_clients_ccs_and_scr,number_of_clients_ccs,number_of_clients_scr
0,2021,1º,Grupo Secundário,Conglomerado,,ABC-BRASIL (conglomerado),,2,3,4,9,26230,24698,3810
1,2021,1º,Grupo Secundário,Conglomerado,,AGIBANK (conglomerado),"54,79",58,140,73,271,1058431,790848,693843


---
## Delivery

---
## Gerar arquivo python
Exemplo

In [62]:
%%writefile ./scripts/testes.pyg
print('hello world')

Writing ./scripts/testes.pyg
